# This is a test for extraction of radiomics features from the patients

In [1]:
import radiomics
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import ipywidgets
import SimpleITK as sitk
import logging


from scipy import stats
from statsmodels.stats.weightstats import DescrStatsW
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, featureextractor, getFeatureClasses

ModuleNotFoundError: No module named 'radiomics'

In [5]:
# Get the PyRadiomics logger (default log-level = INFO)
logger = radiomics.logger
logger.setLevel(logging.DEBUG)  # set level to DEBUG to include debug log messages in log file

# Write out all log entries to a file
handler = logging.FileHandler(filename='../testLog.txt', mode='w')
formatter = logging.Formatter('%(levelname)s:%(name)s: %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [6]:
folder_path = "../study"

Parameters for radiomcis feature extracion

In [7]:
params = folder_path + "/static_files/radiomics_params.yaml"

## Print some slices of patient 2

In [8]:
subj = "VNSLC_02"
subj_path = "%s/subjects/%s" % (folder_path, subj)
subj_path_freesurf = "%s/freesurfer/%s" % (folder_path, subj)

# image_path = "%s/registration/%s_T1_brain_reg.nii.gz" % (subj_path, subj)
# mask_path = "%s/registration/aparc+aseg_reg.mgz" % (subj_path)

image_path = "%s/dMRI/microstructure/dti/%s_FA.nii.gz" % (subj_path, subj)
mask_path = "%s/dlabel/diff/aparc+aseg+thalnuc.bbr.nii.gz" % (subj_path_freesurf)

image = sitk.ReadImage(image_path)
mask = sitk.ReadImage(mask_path)

# I didnt get this passage but it works
image = sitk.GetImageFromArray(sitk.GetArrayFromImage(image))
mask = sitk.GetImageFromArray(sitk.GetArrayFromImage(mask))

In [61]:
# bb, correctedMask = imageoperations.checkMask(image, mask, label=10)
# if correctedMask is not None:
#     mask = correctedMask
# croppedImage, croppedMask = imageoperations.cropToTumorMask(image, mask, bb)
# 
# plt.imshow(sitk.GetArrayFromImage(croppedImage)[10, :, :], cmap="gray")
# plt.imshow(np.where(sitk.GetArrayFromImage(croppedMask)[10, :, :] == 10, 1, 0), alpha=0.5)
# plt.show()

In [62]:
print(image[60, 60, 30])

0.8405095338821411


## Load on pyRadiomics


In [63]:
extractor = featureextractor.RadiomicsFeatureExtractor(params)

# extractor.enableAllImageTypes()
# extractor.enableAllFeatures()

# ----- Test the mean, standard deviaton, skewness and kurtosis
extractor.disableAllFeatures()
extractor.enableFeaturesByName(firstorder=['Mean', 'StandardDeviation', 'Skewness', 'Kurtosis'])

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)
print('Enabled features:\n\t', extractor.enabledFeatures)

Extraction parameters:
	 {'minimumROIDimensions': 3, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkNearestNeighbor', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'weightingNorm': 'euclidean'}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'firstorder': ['Mean', 'StandardDeviation', 'Skewness', 'Kurtosis']}


In [64]:
featureClasses = getFeatureClasses()
print('Active features:')
for cls, features in extractor.enabledFeatures.items():
    if len(features) == 0:
        features = [f for f, deprecated in featureClasses[cls].getFeatureNames().items() if not deprecated]
    for f in features:
        print(f)
        print(getattr(featureClasses[cls], 'get%sFeatureValue' % f).__doc__)

Active features:
Mean

    **8. Mean**

    .. math::
      \textit{mean} = \frac{1}{N_p}\displaystyle\sum^{N_p}_{i=1}{\textbf{X}(i)}

    The average gray level intensity within the ROI.
    
StandardDeviation

    **15. Standard Deviation**

    .. math::
      \textit{standard deviation} = \sqrt{\frac{1}{N_p}\sum^{N_p}_{i=1}{(\textbf{X}(i)-\bar{X})^2}}

    Standard Deviation measures the amount of variation or dispersion from the Mean Value. By definition,
    :math:`\textit{standard deviation} = \sqrt{\textit{variance}}`

    .. note::
      As this feature is correlated with variance, it is marked so it is not enabled by default.
      To include this feature in the extraction, specify it by name in the enabled features
      (i.e. this feature will not be enabled if no individual features are specified (enabling 'all' features),
      but will be enabled when individual features are specified, including this feature).
      Not present in IBSI feature definitions (correlated wit

In [65]:
result = extractor.execute(image, mask, label=13)

Feature StandardDeviation is deprecated, use with caution!


In [67]:
print('Calculated features')
for key, value in result.items():
    print('\t', key, ':', value)

Calculated features
	 diagnostics_Versions_PyRadiomics : 3.1.0a2
	 diagnostics_Versions_Numpy : 1.25.2
	 diagnostics_Versions_SimpleITK : 2.2.1
	 diagnostics_Versions_PyWavelet : 1.4.1
	 diagnostics_Versions_Python : 3.9.18
	 diagnostics_Configuration_Settings : {'minimumROIDimensions': 3, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkNearestNeighbor', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 13, 'additionalInfo': True, 'weightingNorm': 'euclidean'}
	 diagnostics_Configuration_EnabledImageTypes : {'Original': {}}
	 diagnostics_Image-original_Hash : dfde70d7892dd931c329721171d553160016d652
	 diagnostics_Image-original_Dimensionality : 3D
	 diagnostics_Image-original_Spacing : (1.0, 1.0, 1.0)
	 diagnostics_Image-original_Size : (110, 110, 68)
	 diagnostics_Image-original_Mean : 0.07767649852646054
	 diagnosti

## Comparison with the metrics compute with nibabel, numpy and scipy

In [47]:
image = nib.load(image_path).get_fdata()
mask = nib.load(mask_path).get_fdata()

mask = np.where(mask == 13, 1, 0) # Take only the thalamus

In [69]:
image[60, 60, 30]

0.8405095338821411

In [70]:
assert image.shape == mask.shape

v = image.ravel()
w = mask.ravel()

assert v.size == w.size

# Mean
print("Mean", np.average(v, weights=w))
# Standard Deviation
dstat = DescrStatsW(v, w)
print("Std", dstat.std)
# Skewness
w_discrete = np.round(w).astype(int)
repeat = np.repeat(v, w_discrete)
print("Skewness", stats.skew(repeat, bias=True))
# Kurtosis
print("Kurtosis", stats.kurtosis(repeat, fisher=False, bias=True))


Mean 0.43695690652780367
Std 0.21170094617504082
Skewness 0.5876355880797967
Kurtosis 2.2932650784888624


# Managing the ROI from tracts

In [42]:
from dipy.io.streamline import load_tractogram, save_tractogram

# trk_path = "%s/dpath/acomm_avg16_syn_bbr/path.pd.trk" % subj_path_freesurf
trk_path = "%s/dMRI/tractography/left-antThalRadiation.trk" % subj_path

trk = load_tractogram(trk_path, "same")
trk.to_vox()
trk.to_corner()

from unravel.stream import smooth_streamlines
out_path = "../prova.trk"
save_tractogram(trk, out_path)
# if the tract comes from MRtrix3 the smoothing is useless ... (only loose time)
# for _ in range(20):
#     smooth_streamlines(out_path, out_path)


True

In [43]:
from unravel.stream import extract_nodes
# from unravel.viz import plot_nodes_and_surfaces

point_array = extract_nodes(out_path)
# plot_nodes_and_surfaces(point_array)

In [38]:
from unravel.stream import remove_outlier_streamlines

remove_outlier_streamlines(out_path, point_array, out_path, outlier_ratio=100)

959 streamlines removed from tract


In [54]:
from unravel.stream import get_roi_sections_from_nodes

ROI = get_roi_sections_from_nodes(out_path, point_array)
ROI = ROI.astype("float64", casting="safe")
nib.save(nib.Nifti1Image(ROI, trk.affine), "../prova.nii.gz")


# Prova tracula da cancellare

In [1]:
import os
from dipy.io.streamline import load_tractogram, save_trk

ModuleNotFoundError: No module named 'dipy'

In [ ]:
def convertTck2Trk(subj_folder_path, subj_id, tck_path):
    if not os.path.isfile(tck_path): # only if there was an error during the tractography, to not block everything
        return
    print("Converting %s" % tck_path)
    tract = load_tractogram(tck_path, subj_folder_path+"/dMRI/preproc/"+subj_id+"_b0_preproc.nii.gz")
    save_trk(tract, tck_path[:-3]+'trk')

In [ ]:
def convertTck2Trk(subj_folder_path, subj_id, tck_path):
    if not os.path.isfile(tck_path): # only if there was an error during the tractography, to not block everything
        return
    print("Converting %s" % tck_path)
    tract = load_tractogram(tck_path, subj_folder_path+"/dMRI/preproc/"+subj_id+"_b0_preproc.nii.gz")
    save_trk(tract, tck_path[:-3]+'trk')

In [ ]:
import os

tck_path_forward = "%s/dMRI/tractography/left-antThalRadiation_from.trk" % subj_path
tck_path_backward = "%s/dMRI/tractography/left-antThalRadiation_to.trk" % subj_path

outlier_radio = 3
cmd = "tckedit -force "

# Remotion sigularly
if os.path.isfile(output_path_forward):
    convertTck2Trk(subj_folder_path, p_code, output_path_forward)
    output_path_forward = output_path_forward[:-3] + 'trk' # rename to the trk
    forward_point_array = extract_nodes(output_path_forward)
    remove_outlier_streamlines(output_path_forward, forward_point_array, output_path_forward, outlier_radio)
    cmd = cmd + output_path_forward + " "
    # convert to tck

if os.path.isfile(output_path_backward):
    convertTck2Trk(subj_folder_path, p_code, output_path_backward)
    output_path_backward = output_path_backward[:-3] + 'trk' # rename to the trk
    backward_point_array = extract_nodes(output_path_backward)
    remove_outlier_streamlines(output_path_backward, backward_point_array, output_path_backward, outlier_radio)
    cmd = cmd + output_path_backward + " "
    # convert to tck

# Union of the track in forward and backward
cmd = cmd + output_path
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
process.wait()
# os.remove(output_path_forward); os.remove(output_path_backward)
# eliminare anche entrambi i tck e trk

# Remotion outlier in both forward and backward
# convert to trk
point_array = extract_nodes(output_path)
remove_outlier_streamlines(output_path, point_array, output_path, outlier_radio)